# Team Assignment 1 
### predict price and minimum night

Team Member: Xiaoying Zou, Xinyue Yu, Yufeng Xie, Yuxin Tang, Shimin Liang

In [4]:
# import the pakages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import adfuller as ADF
from statsmodels.tsa.stattools import kpss as KPSS
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox as box
# import pmdarima as pmd
# from pmdarima import model_selection
from sklearn.metrics import mean_squared_error
from sklearn.metrics import  mean_absolute_percentage_error
import seaborn as sns
import itertools
from statsmodels.tsa.stattools import arma_order_select_ic as ORDER
from plotnine import * 
from statsmodels.tsa.api import ExponentialSmoothing as es
import statsmodels.formula.api as smf
import calendar
# from pycaret.time_series import *
import xgboost as xgb
import lightgbm as lgb
import plotly.express as px


import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.api import ExponentialSmoothing # For exponential smoothing

c:\Users\Olivia\Anaconda3\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
c:\Users\Olivia\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
c:\Users\Olivia\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.Float64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
c:\Users\Olivia\Anaconda3\lib\site-packages\dask\dataframe\utils.py:369: FutureWarning: pandas.UInt64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.


In [5]:
# Dataset
df = pd.read_csv('dfc.csv')
df.head()

,date,index,id,Unnamed: 0,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,...,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,Scrape File,lm_review,month_review
0,2015-05-01,0,109,708,Amazing bright apartment parkfront,521,Paolo,NaN,Culver City,33.982095,...,146,7,1,2011-08-15,0.00,1,274.0,los_angeles_2015_05_listings.csv,NaN,NaN
1,2015-07-01,1,109,5637,Amazing bright apartment parkfront,521,Paolo,NaN,Culver City,33.982095,...,85,7,1,2011-08-15,0.02,1,336.0,los_angeles_2015_07_listings.csv,1.0,0.0
2,2015-09-01,2,109,1977,Amazing bright apartment parkfront,521,Paolo,NaN,Culver City,33.982095,...,112,7,1,2011-08-15,0.02,1,283.0,los_angeles_2015_09_listings.csv,1.0,0.0
3,2015-11-01,3,109,2106,Amazing bright apartment parkfront,521,Paolo,NaN,Culver City,33.982095,...,111,7,1,2011-08-15,0.02,1,348.0,los_angeles_2015_11_listings.csv,1.0,0.0
4,2016-01-01,4,109,2378,Amazing bright apartment parkfront,521,Paolo,NaN,Culver City,33.982095,...,117,7,1,2011-08-15,0.02,1,272.0,los_angeles_2016_01_listings.csv,1.0,0.0


In [6]:
# Data preprocess
df['Date'] = pd.to_datetime(df['date'] )
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['YM'] = pd.to_datetime(df['Date']).dt.strftime('%Y-%m')

df=df.loc[df['Date']>'2018-03-01',:]
df.loc[df['Date']>='2020-03-01','covid']=1
df.loc[df['Date']<'2020-03-01','covid']=0
# df=df.dropna(subset=['month_review'])

df.head()


,date,index,id,Unnamed: 0,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,...,calculated_host_listings_count,availability_365,Scrape File,lm_review,month_review,Date,Year,Month,YM,covid
12,2018-04-01,12,109,0,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.982095,...,1,0.0,los_angeles_2018_04_listings.csv,2.0,0.0,2018-04-01,2018,4,2018-04,0.0
13,2018-05-01,13,109,0,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.982095,...,1,25.0,los_angeles_2018_05_listings.csv,2.0,0.0,2018-05-01,2018,5,2018-05,0.0
14,2018-06-01,14,109,0,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.982095,...,1,52.0,los_angeles_2018_06_listings.csv,2.0,0.0,2018-06-01,2018,6,2018-06,0.0
15,2018-07-01,15,109,0,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.982095,...,1,82.0,los_angeles_2018_07_listings.csv,2.0,0.0,2018-07-01,2018,7,2018-07,0.0
16,2018-08-01,16,109,0,Amazing bright elegant condo park front *UPGRA...,521,Paolo,Other Cities,Culver City,33.982095,...,1,114.0,los_angeles_2018_08_listings.csv,2.0,0.0,2018-08-01,2018,8,2018-08,0.0


## predict numerical features

In [7]:
df.columns

Index(['date', 'index', 'id', 'Unnamed: 0', 'name', 'host_id', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'latitude', 'longitude',
       'room_type', 'price', 'minimum_nights', 'number_of_reviews',
       'last_review', 'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'Scrape File', 'lm_review', 'month_review', 'Date',
       'Year', 'Month', 'YM', 'covid'],
      dtype='object')

In [8]:
n_group=list(df['neighbourhood_group'].drop_duplicates())
room_type=list(df['room_type'].drop_duplicates())
room_type.remove('Hotel room')

In [9]:
new_month=['2021-02-01','2021-03-01','2021-04-01','2021-05-01','2021-06-01','2021-07-01','2021-08-01','2021-09-01','2021-10-01','2021-11-01','2021-12-01',
    '2022-01-01','2022-02-01','2022-03-01','2022-04-01','2022-05-01','2022-06-01','2022-07-01','2022-08-01','2022-09-01','2022-10-01','2022-11-01','2022-12-01',
    '2023-01-01','2023-02-01','2023-03-01','2023-04-01','2023-05-01','2023-06-01']

In [10]:
attribute=df.loc[(df['room_type']=='Entire home/apt')&(df['neighbourhood_group']=='Other Cities'),['Date','price','minimum_nights']].groupby(['Date']).median()
attribute

,price,minimum_nights
Date,,
2018-04-01,164.0,2.0
2018-05-01,162.0,2.0
2018-06-01,160.0,2.0
2018-07-01,160.0,2.0
2018-08-01,159.0,2.0
2018-09-01,159.0,2.0
2018-10-01,155.0,2.0
2018-11-01,155.0,2.0
2018-12-01,150.0,2.0


In [11]:
output_attribute=pd.DataFrame()


In [12]:
for i in n_group:
    for j in room_type:
        print(i,j)
        attribute=df.loc[(df['room_type']==j)&(df['neighbourhood_group']==i),['Date','price','minimum_nights']].groupby(['Date']).median()
        attribute['Date']=attribute.index

        # Build exponential smoothing model and model trend and multiplictive seasonality
        #hotel room 不够长
        M1= ExponentialSmoothing(attribute['price'],trend='add',seasonal='mul',seasonal_periods=12).fit() # add means addictive trend
        print(M1.summary())
        attribute['M1']=M1.fittedvalues

        M2= ExponentialSmoothing(attribute['minimum_nights'],trend='add',seasonal='mul',seasonal_periods=12).fit() # add means addictive trend
        print(M2.summary())
        attribute['M2']=M2.fittedvalues


    


        for m in new_month:
            attribute.loc[m,:]=np.nan
        attribute.loc['2021-02-01':'2023-06-01','price']=M1.forecast(29) 
        attribute.loc['2021-02-01':'2023-06-01','minimum_nights']=M2.forecast(29) 

        attribute['room_type']=j
        attribute['neighbourhood_group']=i
        # print(attribute.head())
        # attribute=attribute['price','minimum_nights','room_type','neighbourhood_group']
        output_attribute=pd.concat([attribute,output_attribute])

output_attribute=output_attribute[['price','minimum_nights','room_type','neighbourhood_group']]


Other Cities Entire home/apt


c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.


                       ExponentialSmoothing Model Results                       
Dep. Variable:                    price   No. Observations:                   34
Model:             ExponentialSmoothing   SSE                             90.062
Optimized:                         True   AIC                             65.121
Trend:                         Additive   BIC                             89.542
Seasonal:                Multiplicative   AICC                           110.721
Seasonal Periods:                    12   Date:                 Wed, 15 Feb 2023
Box-Cox:                          False   Time:                         14:14:12
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               0.7822218                alpha                 True
smoothing_trend          

c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.


                       ExponentialSmoothing Model Results                       
Dep. Variable:                    price   No. Observations:                   34
Model:             ExponentialSmoothing   SSE                             51.857
Optimized:                         True   AIC                             46.352
Trend:                         Additive   BIC                             70.774
Seasonal:                Multiplicative   AICC                            91.952
Seasonal Periods:                    12   Date:                 Wed, 15 Feb 2023
Box-Cox:                          False   Time:                         14:14:13
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               0.6781999                alpha                 True
smoothing_trend          

...

Unincorporated Areas Shared room
                       ExponentialSmoothing Model Results                       
Dep. Variable:                    price   No. Observations:                   34
Model:             ExponentialSmoothing   SSE                            111.467
Optimized:                         True   AIC                             72.371
Trend:                         Additive   BIC                             96.792
Seasonal:                Multiplicative   AICC                           117.971
Seasonal Periods:                    12   Date:                 Wed, 15 Feb 2023
Box-Cox:                          False   Time:                         14:14:15
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               1.0000000                alpha              

c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.


                       ExponentialSmoothing Model Results                       
Dep. Variable:           minimum_nights   No. Observations:                   34
Model:             ExponentialSmoothing   SSE                              3.384
Optimized:                         True   AIC                            -46.451
Trend:                         Additive   BIC                            -22.029
Seasonal:                Multiplicative   AICC                            -0.851
Seasonal Periods:                    12   Date:                 Wed, 15 Feb 2023
Box-Cox:                          False   Time:                         14:14:15
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               1.0000000                alpha                 True
smoothing_trend          

c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.
c:\Users\Olivia\Anaconda3\lib\site-packages\statsmodels\tsa\base\tsa_model.py:471: ValueWarning: No frequency information was provided, so inferred frequency MS will be used.


                       ExponentialSmoothing Model Results                       
Dep. Variable:                    price   No. Observations:                   34
Model:             ExponentialSmoothing   SSE                             31.415
Optimized:                         True   AIC                             29.312
Trend:                         Additive   BIC                             53.733
Seasonal:                Multiplicative   AICC                            74.912
Seasonal Periods:                    12   Date:                 Wed, 15 Feb 2023
Box-Cox:                          False   Time:                         14:14:16
Box-Cox Coeff.:                    None                                         
                          coeff                 code              optimized      
---------------------------------------------------------------------------------
smoothing_level               0.9389332                alpha                 True
smoothing_trend          

In [ ]:
output_attribute.to_csv('predicted_feature.csv')